In [4]:
import streamlit as st
import requests
import geopandas as gpd
import pandas as pd
import folium
from folium import IFrame
from shapely.geometry import shape
from streamlit_folium import st_folium
import branca.colormap as cm
import math

API_KEY = "ea9637fd9f0c41f3e2e932faa99dfcd76f8041aa"

2025-02-16 12:47:20.793 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
def fetch_state_data():
    """Fetch state-level Census data, return as DataFrame."""
    state_url = (
        f"https://api.census.gov/data/2019/pep/population"
        f"?get=NAME,POP&for=state:*&key={API_KEY}"
    )
    r_states = requests.get(state_url)
    state_data = r_states.json()
    df = pd.DataFrame(state_data[1:], columns=state_data[0])
    df["POP"] = df["POP"].astype(int)
    return df

In [5]:
state_data = fetch_state_data()
state_data.head()

,NAME,POP,state
0,Alabama,4903185,01
1,Alaska,731545,02
2,Arizona,7278717,04
3,Arkansas,3017804,05
4,California,39512223,06


In [20]:
def fetch_county_data():
    """Fetch county-level Census data, return as DataFrame."""
    county_url = (
        f"https://api.census.gov/data/2019/pep/population"
        f"?get=NAME,POP&for=county:*&key={API_KEY}"
    )
    r_counties = requests.get(county_url)
    county_data = r_counties.json()
    df = pd.DataFrame(county_data[1:], columns=county_data[0])
    df["POP"] = df["POP"].astype(int)
    # Split the NAME column into state and county
    df[['countyName', 'stateName']] = df['NAME'].str.split(',', expand=True)
    # Combine state and county to form FIPS
    df["FIPS"] = df["state"].str.zfill(2) + df["county"].str.zfill(3)
    return df

In [21]:
county_data = fetch_county_data()
county_data.head()

,NAME,POP,state,county,countyName,stateName,FIPS
0,"Fayette County, Illinois",21336,17,051,Fayette County,Illinois,17051
1,"Logan County, Illinois",28618,17,107,Logan County,Illinois,17107
2,"Saline County, Illinois",23491,17,165,Saline County,Illinois,17165
3,"Massac County, Illinois",13772,17,127,Massac County,Illinois,17127
4,"Cass County, Illinois",12147,17,017,Cass County,Illinois,17017


In [22]:
def fetch_geojson(url: str):
    """Cache the geojson data from the provided URL."""
    return requests.get(url).json()

In [23]:
state_abbrev_to_fips = {
    "AL": "01", "AK": "02", "AZ": "04", "AR": "05", "CA": "06", "CO": "08",
    "CT": "09", "DE": "10", "DC": "11", "FL": "12", "GA": "13", "HI": "15",
    "ID": "16", "IL": "17", "IN": "18", "IA": "19", "KS": "20", "KY": "21",
    "LA": "22", "ME": "23", "MD": "24", "MA": "25", "MI": "26", "MN": "27",
    "MS": "28", "MO": "29", "MT": "30", "NE": "31", "NV": "32", "NH": "33",
    "NJ": "34", "NM": "35", "NY": "36", "NC": "37", "ND": "38", "OH": "39",
    "OK": "40", "OR": "41", "PA": "42", "RI": "44", "SC": "45", "SD": "46",
    "TN": "47", "TX": "48", "UT": "49", "VT": "50", "VA": "51", "WA": "53",
    "WV": "54", "WI": "55", "WY": "56"
}

In [24]:
def build_states_gdf(state_df, state_abbrev_to_fips):
    """Build GeoDataFrame for US states."""
    state_pop_dict = state_df.set_index("state")["POP"].to_dict()
    state_name_dict = state_df.set_index("state")["NAME"].to_dict()
    url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"
    geo_data = fetch_geojson(url)
    rows = []
    for feat in geo_data["features"]:
        abbrev = feat["id"]
        geom = shape(feat["geometry"])
        fips = state_abbrev_to_fips.get(abbrev)
        if fips:
            pop_val = state_pop_dict.get(fips, "No data")
            name_val = state_name_dict.get(fips, "No data")
            rows.append({
                "geometry": geom,
                "STATE_FIPS": fips,
                "NAME": name_val,
                "POP": pop_val
            })
    return gpd.GeoDataFrame(rows, crs="EPSG:4326")

In [25]:
state_gdf = build_states_gdf(state_data, state_abbrev_to_fips)

In [26]:
state_gdf.head()

,geometry,STATE_FIPS,NAME,POP
0,"POLYGON ((-87.3593 35.00118, -85.60668 34.9847...",01,Alabama,4903185
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",02,Alaska,731545
2,"POLYGON ((-109.0425 37.00026, -109.04798 31.33...",04,Arizona,7278717
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",05,Arkansas,3017804
4,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",06,California,39512223


In [27]:
def build_counties_gdf(county_df):
    """Build GeoDataFrame for US counties and simplify geometries for performance."""
    county_pop_dict = county_df.set_index("FIPS")["POP"].to_dict()
    county_name_dict = county_df.set_index("FIPS")["countyName"].to_dict()
    state_name_dict = county_df.set_index("FIPS")["stateName"].to_dict()
    name_dict = county_df.set_index("FIPS")["NAME"].to_dict()
    url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
    geo_data = fetch_geojson(url)
    rows = []
    for feat in geo_data["features"]:
        fips = feat["id"]
        geom = shape(feat["geometry"])
        pop_val = county_pop_dict.get(fips, "No data")
        county_name_val = county_name_dict.get(fips, "No data") 
        state_name_val = state_name_dict.get(fips, "No data")
        name_val = name_dict.get(fips, "No data")
        rows.append({
            "geometry": geom,
            "FIPS": fips,
            "COUNTY_NAME": county_name_val,
            "STATE_NAME": state_name_val,
            "NAME": name_val,
            "POP": pop_val
        })
    gdf = gpd.GeoDataFrame(rows, crs="EPSG:4326")
    gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.01, preserve_topology=True)
    return gdf

In [28]:
counties_gdf = build_counties_gdf(county_data)
counties_gdf.head()

,geometry,FIPS,COUNTY_NAME,STATE_NAME,POP
0,"POLYGON ((-86.49677 32.34444, -86.7179 32.4028...",01001,Autauga County,Alabama,55869
1,"POLYGON ((-86.5778 33.76532, -86.75914 33.8406...",01009,Blount County,Alabama,57826
2,"POLYGON ((-85.18413 32.87052, -85.12342 32.772...",01017,Chambers County,Alabama,33254
3,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",01021,Chilton County,Alabama,44428
4,"POLYGON ((-88.13999 34.5817, -88.09789 34.8922...",01033,Colbert County,Alabama,55241
